In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from helper import *
from stories import *
from pipelines import *
import psycopg2

In [2]:
now = datetime.now()
date_time = now.strftime("%Y%m%d")

## OpenAI

In [3]:
set_api_key()

## Connect to Database

In [4]:
conn = get_connection()

## Load Projects

In [5]:
projects = get_useful_projects()

## Functions

In [6]:
def save_in_database(conn, prompt1, prompt2, encoding, story, story_1, story_2, version, file):
    cursor = conn.cursor()
    command = "INSERT INTO ne_linebyline (prompt1, prompt2, encoding, story, story_1, story_2, version, file) VALUES ('%(prompt1)s', '%(prompt2)s', '%(encoding)s', '%(story)s', '%(story_1)s', '%(story_2)s', '%(version)s', '%(file)s');" % {"prompt1": prompt1, "prompt2": prompt2, "encoding": encoding, "story": story, "story_1": story_1, "story_2":story_2, "version":version, "file":file}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=}")
        conn.commit()
        cursor.close()
        pass

In [7]:
def quality_check(name_file, lines_file, previous_messages, instance, deep):
    if deep==0:
        return lines_file[-1]
    with open(name_file,"w") as tmp_file:
        for new_line in lines_file:
            tmp_file.write(new_line)
    models, errors, symbols, mistakes = asp_try_5(name_file,instance)
    print(f"Errors {errors}. Messages {mistakes}.")
    new_answer = lines_file[-1]
    if len(mistakes) > 0:
        print("Mistake! Last Message : " + str(previous_messages[-1]))
        answer = get_completion_line("The last rule that you wrote is wrong. The error message is: ###" + str(mistakes) + "###. Can you try again?"
                                    , previous_messages = previous_messages)
        lines_file[-1] = answer
        print("Deep: "+ str(deep) + ". New answer: " +answer)
        new_answer = quality_check(name_file,lines_file,previous_messages, instance, deep-1)
    return lines_file[-1]

## Main : From Script, Line4Line

In [9]:
for project in projects:
    if project in [minotaur]:
        continue
    story = project["story"]
    encoding = project["lines"]
    instance = instances_dict[story]
    problem = project["problem"]
    lines = encoding.splitlines()
    new_file = []
    messages = [{"role":"user","content":f"Problem: {problem}"}]
    for line in lines:
        if line == '\n':
            continue
        if len(line.strip()) > 0 and line.strip()[0] == '%':
            print("---------------------------------------------------")
            new_file.append("\n" + line)
            messages.append({"role":"user","content":line})
            print(line)
            answer = get_completion_line(line,previous_messages=messages)
            print(answer)
            new_file.append("\n" + answer)
            messages.append({"role":"assistant","content":answer})
            #new_answer = quality_check(x",new_file,messages,instance,deep=3)
            #new_file[-1] = new_answer
    new_file_name = story + "_v11_SL_" + date_time + ".lp"
    prompt = '''Given the description of a problem, and a commented line, 
                        your task is to translate this commented line in Answer Set Programming. 
                        Just write one or a few predicates or rules. Do not try to solve the whole problem at once.'''
    encoding=' '.join(new_file)
    save_in_database(conn,prompt,'',encoding.replace("'",""),story,'', '', 11, new_file_name)  

---------------------------------------------------
% Four atoms with predicate "plus" of 2 variables with possible values 0 and 1.
plus(0,0).
plus(0,1).
plus(1,0).
plus(1,1).
---------------------------------------------------
% The four possible directions, predicate "dir" with 2 variables, of value 0, 1 or -1. dir(0,0) is not an option.
dir(0,1).
dir(0,-1).
dir(1,0).
dir(-1,0).
---------------------------------------------------
% Predicate "black" of 2 variables is generated as the sum of the sum of the position (X,Y) and (A,B), for plus (A,B), if the cell(X+A,Y+B) exists, given hint(X,Y,N). There should be exactly "N" "black" predicates generated.
{ black(X+DX,Y+DY) : dir(DX,DY), cell(X+DX,Y+DY) } = N :- hint(X,Y,N).
---------------------------------------------------
% Predicate "path" has 4 variables, indicating a path between two cells of predicate "white" of positions (X,Y) and (P,Q) if the sum of the absolute values of the differences of P and X, and Q and Y, is 1. Absolute v

In [10]:
new_file

['% Four atoms with predicate "plus" of 2 variables with possible values 0 and 1.',
 'plus(0,0).\nplus(0,1).\nplus(1,0).\nplus(1,1).',
 '% The four possible directions, predicate "dir" with 2 variables, of value 0, 1 or -1. dir(0,0) is not an option.',
 'dir(0,1).\ndir(0,-1).\ndir(1,0).\ndir(-1,0).',
 '% Predicate "black" of 2 variables is generated as the sum of the sum of the position (X,Y) and (A,B), for plus (A,B), if the cell(X+A,Y+B) exists, given hint(X,Y,N). There should be exactly "N" "black" predicates generated.',
 '{ black(X1,Y1) : dir(A,B), cell(X+A,Y+B), plus(A,B) } = N :- hint(X,Y,N).',
 '% Predicate "path" has 4 variables, indicating a path between two cells of predicate "white" of positions (X,Y) and (P,Q) if the sum of the absolute values of the differences of P and X, and Q and Y, is 1. Absolute value is expresed with bars: "||"',
 'path(X,Y,P,Q) :- white(X,Y), white(P,Q), |P-X| + |Q-Y| = 1.',
 '% If a cell is not black, it should be white',
 'white(X,Y) :- cell(X,Y)